In [1]:
import os
import numpy as np
import math
from PIL import Image
import cv2
import matplotlib.pyplot as plt

image1_path = os.getenv('IMAGE1_PATH')
image2_path = os.getenv('IMAGE2_PATH')
output_path = os.getenv('OUTPUT_PATH')

In [ ]:
def resize_image(image, new_width):
    aspect_ratio = image.shape[1] / image.shape[0]
    new_height = int(new_width / aspect_ratio)
    resized_image = cv2.resize(image, (new_width, new_height))
    return resized_image

def build_gaussian_pyramid(image, num_levels):
    gaussian_pyramid = [image]
    for _ in range(1, num_levels):
        image = cv2.pyrDown(image)
        gaussian_pyramid.append(image)
    return gaussian_pyramid

def build_laplacian_pyramid(gaussian_pyramid):
    laplacian_pyramid = []
    num_levels = len(gaussian_pyramid)
    for i in range(num_levels - 1):
        src_size = (gaussian_pyramid[i].shape[1], gaussian_pyramid[i].shape[0])
        dst_size = ((src_size[0] * 2) // 2, (src_size[1] * 2) // 2)  # Compute destination size as half of double size
        gaussian_expanded = cv2.pyrUp(gaussian_pyramid[i + 1], dstsize=dst_size)
        laplacian = cv2.subtract(gaussian_pyramid[i], gaussian_expanded)
        laplacian_pyramid.append(laplacian)
    laplacian_pyramid.append(gaussian_pyramid[-1])
    return laplacian_pyramid


def blend_pyramids(laplacian_pyramid1, laplacian_pyramid2):
    blended_pyramid = []
    for l1, l2 in zip(laplacian_pyramid1, laplacian_pyramid2):
        blended = cv2.addWeighted(l1, 0.5, l2, 0.5, 0)
        blended_pyramid.append(blended)
    return blended_pyramid

def reconstruct_from_pyramid(laplacian_pyramid):
    image = laplacian_pyramid[-1]
    for laplacian in reversed(laplacian_pyramid[:-1]):
        size = (laplacian.shape[1], laplacian.shape[0])
        image = cv2.pyrUp(image, dstsize=size)
        image = cv2.add(image, laplacian)
    return image

In [ ]:
def process_images_1(image1_path, image2_path, output_path):
    
    damaged_face = cv2.imread(image1_path)
    complete_face = cv2.imread(image2_path)
    

    # Get the dimensions to resize to (the smallest of the two images)
    new_height = min(damaged_face.shape[0], complete_face.shape[0])
    new_width = min(damaged_face.shape[1], complete_face.shape[1])

    # Resize both images to the smallest dimensions
    resized_damaged_face = cv2.resize(damaged_face, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    resized_complete_face = cv2.resize(complete_face, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    
    new_width = 500  # Choose a common width
    damaged_image = resize_image(resized_damaged_face, new_width)
    complete_image = resize_image(resized_complete_face, new_width)

    # Build pyramids
    num_levels = 5  # Number of levels in the pyramid
    damaged_gaussian_pyramid = build_gaussian_pyramid(damaged_image, num_levels)
    complete_gaussian_pyramid = build_gaussian_pyramid(complete_image, num_levels)

    damaged_laplacian_pyramid = build_laplacian_pyramid(damaged_gaussian_pyramid)
    complete_laplacian_pyramid = build_laplacian_pyramid(complete_gaussian_pyramid)

    # Blend pyramids
    blended_laplacian_pyramid = blend_pyramids(damaged_laplacian_pyramid, complete_laplacian_pyramid)

    # Reconstruct the image from the blended pyramid
    output = reconstruct_from_pyramid(blended_laplacian_pyramid)
    
    # Save the output image
    cv2.imwrite(output_path, output)
    
    # Optionally, you can return the path to the output image
    return output_path


# Execute the processing function
process_images_1(image1_path, image2_path, output_path)
    